In [447]:
#base_packs
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn.preprocessing as skp
import sklearn.decomposition as skd
import statsmodels.api as sm
import sklearn.manifold as skm
import sklearn.cluster as skc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from statsmodels.stats.diagnostic import het_white
import ruptures as rpt
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report
import statsmodels.api as sm
import sklearn.preprocessing as skp
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
import statsmodels.discrete.discrete_model as sm
from sklearn.cluster import KMeans
import gc
import warnings
from scipy.optimize import OptimizeWarning
import time
import numpy as np
import random
from sklearn.preprocessing import RobustScaler, QuantileTransformer,MinMaxScaler
from sklearn.metrics import calinski_harabasz_score
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import GaussianNB
import warnings

In [448]:
dt = 'V:/Departamentos/DVM/Transformação Digital/Projetos/064 - Otimização de Processo com IA no VOD (Ciência de Dados)/VOD_JUL_2024.xlsx'
dt = pd.read_excel(dt)
dt = pd.DataFrame(dt)
dt['Tempo de depressão de vácuo(min)'].replace(',' , '.', inplace = True)
dt['Tempo de depressão de vácuo(min)']= pd.to_numeric(dt['Tempo de depressão de vácuo(min)'] , errors = 'coerce')

dt['Teor de C após a descarburação(%)'].replace(',' , '.', inplace = True)
dt['Teor de C após a descarburação(%)']= pd.to_numeric(dt['Teor de C após a descarburação(%)'] , errors = 'coerce')
dt.drop(columns=['DATA', 'Agitação média do TruStir durante o sopro de O2','Agitação média do TruStir durante a descarburação'], inplace=True)
dt.replace('Boa', 1 , inplace = True)
dt.replace('Ruim', 0 , inplace = True)
dt = pd.get_dummies(dt)
for i in dt.columns:
    median_value = dt[i].median()  
    dt[i].fillna(median_value, inplace=True)
dt = dt.replace({False: 0, True: 1})

In [449]:
teste = 'C:/Users/S409478/Documents/vm_report_vo1727442932637.xls'
teste = pd.read_excel(teste)
teste = pd.DataFrame(teste)
# Extract the first row and set as new header
new_header = teste.iloc[0]
teste = teste[1:]  # Drop the first row
teste.columns = new_header  # Set new header

# Reset index
teste.reset_index(drop=True, inplace=True)

# Function to format numbers
def insert_decimal(x):
    x_str = str(x)
    
    # Handle cases where there is already a decimal point
    if '.' in x_str:
        integer_part, decimal_part = x_str.split('.')
        # If the integer part is shorter than 2 digits, just return as is
        if len(integer_part) <= 3:
            return x_str
        # Otherwise, insert decimal point after the second-to-last digit
        return integer_part[:-3] + '.' + integer_part[-3:] + (f".{decimal_part}" if decimal_part else "")
    
    # Handle integer values
    if len(x_str) > 3:
        return x_str[:-3] + '.' + x_str[-3:]
    return x_str

# Apply the function to each element in the DataFrame column
teste['Tempo abaixo de 10 mbar(min)'] = teste['Tempo abaixo de 10 mbar(min)'].apply(insert_decimal)
#teste['Tempo de depressão de vácuo(min)'].replace(',' , '.', inplace = True)
teste.drop(columns=['DATA', 'Agitação média do TruStir durante o sopro de O2','Agitação média do TruStir durante a descarburação','AÇO'], inplace=True)
#teste = pd.to_numeric(teste, errors='coerce')
teste = teste.apply(pd.to_numeric, errors='coerce')
teste = teste.dropna(how = 'any')
teste = teste.reset_index(drop=True)

In [450]:
#priorizar o teor de c após a descarburação na otimização.
# optimização:
dt['opt'] = 0
dt.loc[
    (dt['Teor de C após a descarburação(%)'] < 0.009) &
(dt['Temperatura do aço após a descarburação(°C)'] >= 1680) &
(dt['Temperatura do aço após a descarburação(°C)'] <= 1700)
    ,
    'opt'
] = 1

#priorizar o teor de c após a descarburação na otimização.
# optimização:
teste['opt'] = 0
teste.loc[
    (teste['Teor de C após a descarburação(%)'] < 0.009) &
(teste['Temperatura do aço após a descarburação(°C)'] >= 1680) &
(teste['Temperatura do aço após a descarburação(°C)'] <= 1700)
    ,
    'opt'
] = 1

In [497]:
#NAIVE_BAYES:
#Primeira volta:
# Initialize the Gaussian Naive Bayes classifier
# Required columns for the model
warnings.filterwarnings('ignore')

required_columns = [
    'Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
    'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
    'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
    'Temperatura do aço antes do VOD(°C)','Peso de aço antes do VOD(kg) ',
    'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela',
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
    'Pressão média de vácuo durante o sopro de O2(mbar)',
    'Tempo abaixo de 10 mbar(min)',
    'Vazão média de argônio durante a descarburação(Nm3/h)',
    'Volume de O2 soprado(m3)']

# Define control DataFrame
columns_control = [
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
    'Pressão média de vácuo durante o sopro de O2(mbar)',
    'Tempo abaixo de 10 mbar(min)',
    'Vazão média de argônio durante a descarburação(Nm3/h)',
    'Volume de O2 soprado(m3)'
]

# Example control values
stp = [[ 5.  , 494.7 ,  18.7 ,   6.54, 194.  ]]
contr = pd.DataFrame(stp, columns=columns_control)

# Prepare input DataFrame
ent = teste[['Teor de C antes do VOD(%)',
       'Teor de Cr antes do VOD(%)', 'Teor de Si antes do VOD(%)',
       'Teor de Mn antes do VOD(%)', 'Teor de Ni antes do VOD(%)','Teor de Mo antes do VOD(%)',
          'Temperatura do aço antes do VOD(°C)',
       'Peso de aço antes do VOD(kg) ', 'Borda livre da panela(mm)',
       'Vida de plug da panela', 'Vida da panela']]

#ent_ = ent.sample(n=1)

# Combine control and input data
contr.reset_index(drop=True, inplace=True)
tt2 = pd.concat([ent_, contr], axis=1)
tt2.fillna(0, inplace=True)
tt2 = pd.DataFrame(tt2.sum()).T
# Predicting on tt2

tt2_features = tt2.values
# Prepare test data for predictions
tt2_test = tt2[columns_control]

# Prepare Data for training
x_ = dt[required_columns]
y_ = dt['opt']
scaler = RobustScaler()
x_scaled = scaler.fit_transform(x_)

# Split the data

gnb = GaussianNB()
#x_tr, x_tes, y_tr, y_tes = train_test_split(x_, y_, test_size=0.2, random_state=42)
# Fit the model on the training data
gnb.fit(x_ , y_)

# Make predictions on the test data
y_pred = gnb.predict(tt2_features)
y_prob = gnb.predict_proba(tt2_features)
print(y_pred, ent_.index, y_prob)

[1] Index([44], dtype='int64') [[0.31127327 0.68872673]]


In [455]:
teste.iloc[ent_.index][columns_control].values

array([[  5.  , 494.7 ,  18.7 ,   6.54, 194.  ]])

In [454]:
teste.iloc[ent_.index][['opt', 'Teor de C após a descarburação(%)','Temperatura do aço após a descarburação(°C)']]

,opt,Teor de C após a descarburação(%),Temperatura do aço após a descarburação(°C)
44,1,0.006,1687.0


In [498]:
## LOGIT/NB:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from sklearn.naive_bayes import GaussianNB
import time
import warnings
import gc

# Custom Huber loss function for logistic regression without L2 regularization
def huber_loss(params, X, y, delta=1.0):
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    loss = np.where(abs_residuals <= delta,
                    0.5 * residuals**2,
                    delta * (abs_residuals - 0.5 * delta))
    loss = np.sum(loss) / n
    
    return loss

def huber_loss_grad(params, X, y, delta=1.0):
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    grad = -np.dot(X.T, np.where(abs_residuals <= delta,
                                 residuals,
                                 delta * np.sign(residuals))) / n

    return grad

# Initialize the loop variables
warnings.filterwarnings('ignore')
condition_met = False
start_time = time.time()
time_limit = 30
iteration = 0

# Define constraints
constraints = {
    'Tempo abaixo de 10 mbar(min)': (9.650573007040464, 23),
    'Vazão média de argônio durante o sopro de O2(Nm3/h)': (3.5215818863059383, 6.8251891508760565),
    'Pressão média de vácuo durante o sopro de O2(mbar)': (400, 693.3164964745301),
    'Volume de O2 soprado(m3)': (115.39118253104017, 248.45226169596572),
    'Vazão média de argônio durante a descarburação(Nm3/h)': (4.55, 7.87)
}

while not condition_met:
    try:
        # Track elapsed time
        elapsed_time = time.time() - start_time
        if elapsed_time > time_limit:
            print("Time limit exceeded. Stopping the loop.")
            break

        iteration += 1
        print(f"Iteration: {iteration}, Time elapsed: {elapsed_time:.2f} seconds")

        # Reset index for contr DataFrame
        contr.reset_index(drop=True, inplace=True)

        # Concatenate and process DataFrames
        tt2 = pd.concat([ent_, contr], axis=1)
        tt2.fillna(0, inplace=True)
        tt2 = pd.DataFrame(tt2.sum()).T
        
        # Apply constraints efficiently
        for col, (a_min, a_max) in constraints.items():
            if col in tt2.columns:
                tt2[col] = np.clip(tt2[col], a_min=a_min, a_max=a_max)

        required_columns = [
            'Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
            'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
            'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
            'Temperatura do aço antes do VOD(°C)', 'Peso de aço antes do VOD(kg) ',
            'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela',
            'Vazão média de argônio durante o sopro de O2(Nm3/h)',
            'Pressão média de vácuo durante o sopro de O2(mbar)',
            'Tempo abaixo de 10 mbar(min)',
            'Vazão média de argônio durante a descarburação(Nm3/h)',
            'Volume de O2 soprado(m3)'
        ]

        tt2_test = tt2[required_columns]

        x_ = dt[required_columns]
        y_ = dt['opt']

        x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_, y_, test_size=0.9)

        # Optimize using Huber loss without L2 regularization
        initial_params = np.zeros(x_train_3.shape[1])
        result = minimize(fun=huber_loss, x0=initial_params, args=(x_train_3.values, y_train_3.values),
                          jac=huber_loss_grad, method='BFGS')

        optimal_params = result.x

        # Predicting on tt2
        tt2_features = tt2_test[required_columns].values
        y_pred_tt2_prob = 1 / (1 + np.exp(-np.dot(tt2_features, optimal_params)))
        y_pred_tt2_binary = (y_pred_tt2_prob >= 0.5).astype(int)

        # Display predictions for tt2
        tt2['Predicted_Probability'] = y_pred_tt2_prob
        tt2['Predicted_Class'] = y_pred_tt2_binary

        print("Predictions on tt2:")
        print(tt2[['Predicted_Probability', 'Predicted_Class']])

        # Train Naive Bayes model on the predicted probabilities
        x_train_nb = np.column_stack((x_train_3.values, 1 / (1 + np.exp(-np.dot(x_train_3.values, optimal_params)))))
        x_test_nb = np.column_stack((x_test_3.values, 1 / (1 + np.exp(-np.dot(x_test_3.values, optimal_params)))))

        nb_model = GaussianNB()
        nb_model.fit(x_, y_)

        # Make predictions using the Naive Bayes model
        y_pred_nb_prob = nb_model.predict_proba(np.column_stack((tt2_test[required_columns].values, y_pred_tt2_prob)))[:, 1]
        y_pred_nb_binary = (y_pred_nb_prob >= 0.5).astype(int)

        # Display Naive Bayes predictions for tt2
        tt2['Naive_Bayes_Probability'] = y_pred_nb_prob
        tt2['Naive_Bayes_Class'] = y_pred_nb_binary

        print("Naive Bayes Predictions on tt2:")
        print(tt2[['Naive_Bayes_Probability', 'Naive_Bayes_Class']])

        # Fit the Logit model from Statsmodels
        logit_model = sm.Logit(y_train_3, x_train_3)
        
        results = logit_model.fit(method='bfgs', cov_type='HC3')

        # Extract coefficients and compute exponentiated coefficients
        co = results.params
        exp_coeff = np.exp(co) / (np.exp(co) + 1)
        exp_coeff_df = pd.DataFrame({'exp_coeff': exp_coeff})

        # Extract results summary
        results_summary = results.summary()
        results_as_html = results_summary.tables[1].as_html()
        df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
        df_results['exp_coeff'] = exp_coeff_df

        # Filter significant variables
        impact_var2 = df_results[df_results['P>|z|'] < 0.05]

        # Calculate results
        results_list = []
        for i in impact_var2.index:
            value = impact_var2.loc[i, 'exp_coeff']
            if i in constraints:
                correction = tt2_test[i].median()
                result = (value - 0.5) * correction if value != 0.5 else 0
                results_list.append({'Column': i, 'Alteração': result})

        results_df_ = pd.DataFrame(results_list).T
        results_df_.columns = results_df_.iloc[0]
        results_df_ = results_df_[1:]

        # Common columns
        common_columns = list(set(constraints.keys()) & set(results_df_.columns))
        contr_common = contr[common_columns]
        results_common = results_df_[common_columns]

        # Combine and update
        combined_df = pd.concat([contr_common, results_common], ignore_index=True)
        final = combined_df.sum().reset_index()
        final_df = final.T
        final_df.columns = final_df.iloc[0]
        final_df = final_df[1:]

        contr.update(final_df[common_columns])
        contr.reset_index(drop=True, inplace=True)

        # Garbage collection
        gc.collect()

        # Check condition
        if np.all(y_pred_nb_prob >= 0.99) and np.all(y_pred_nb_binary == 1):
            condition_met = True

    except Exception as e:
        print(f"An error occurred: {e}")
        continue


Iteration: 1, Time elapsed: 0.00 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0                0.69157                1
An error occurred: X has 17 features, but GaussianNB is expecting 16 features as input.
Iteration: 2, Time elapsed: 0.03 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0           1.230379e-79                0
An error occurred: X has 17 features, but GaussianNB is expecting 16 features as input.
Iteration: 3, Time elapsed: 0.05 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0               0.350672                0
An error occurred: X has 17 features, but GaussianNB is expecting 16 features as input.
Iteration: 4, Time elapsed: 0.08 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0               0.284477                0
An error occurred: X has 17 features, but GaussianNB is expecting 16 features as input.
Iteration: 5, Time elapsed: 0.10 seconds
Predictions on tt2:
   Pred

In [602]:
#NAIVE_BAYES:
#Primeira volta:
# Initialize the Gaussian Naive Bayes classifier
# Required columns for the model
warnings.filterwarnings('ignore')

required_columns = [
    'Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
    'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
    'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
    'Temperatura do aço antes do VOD(°C)','Peso de aço antes do VOD(kg) ',
    'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela',
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
    'Pressão média de vácuo durante o sopro de O2(mbar)',
    'Tempo abaixo de 10 mbar(min)',
    'Vazão média de argônio durante a descarburação(Nm3/h)',
    'Volume de O2 soprado(m3)']

# Define control DataFrame
columns_control = [
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
    'Pressão média de vácuo durante o sopro de O2(mbar)',
    'Tempo abaixo de 10 mbar(min)',
    'Vazão média de argônio durante a descarburação(Nm3/h)',
    'Volume de O2 soprado(m3)'
]

# Example control values
stp = [[ 5.  , 494.7 ,  18.7 ,   6.54, 194.  ]]
contr = pd.DataFrame(stp, columns=columns_control)

# Prepare input DataFrame
ent = teste[['Teor de C antes do VOD(%)',
       'Teor de Cr antes do VOD(%)', 'Teor de Si antes do VOD(%)',
       'Teor de Mn antes do VOD(%)', 'Teor de Ni antes do VOD(%)','Teor de Mo antes do VOD(%)',
          'Temperatura do aço antes do VOD(°C)',
       'Peso de aço antes do VOD(kg) ', 'Borda livre da panela(mm)',
       'Vida de plug da panela', 'Vida da panela']]

#ent_ = ent.sample(n=1)

# Combine control and input data
contr.reset_index(drop=True, inplace=True)
tt2 = pd.concat([ent_, contr], axis=1)
tt2.fillna(0, inplace=True)
tt2 = pd.DataFrame(tt2.sum()).T
# Predicting on tt2

tt2_features = tt2.values
# Prepare test data for predictions
tt2_test = tt2[columns_control]

# Prepare Data for training
x_ = dt[required_columns]
y_ = dt['opt']
scaler = RobustScaler()
x_scaled = scaler.fit_transform(x_)

# Split the data

gnb = GaussianNB()
#x_tr, x_tes, y_tr, y_tes = train_test_split(x_, y_, test_size=0.2, random_state=42)
# Fit the model on the training data
gnb.fit(x_ , y_)

# Make predictions on the test data
y_pred = gnb.predict(tt2_features)
y_prob = gnb.predict_proba(tt2_features)
print(y_pred, ent_.index, y_prob)

[0] Index([39], dtype='int64') [[0.61593032 0.38406968]]


In [607]:
## WITH NAIVE BAYES, OKK


import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from sklearn.naive_bayes import GaussianNB
import time
import warnings
import gc

# Custom Huber loss function for logistic regression without L2 regularization
def huber_loss(params, X, y, delta=1.0):
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    loss = np.where(abs_residuals <= delta,
                    0.5 * residuals**2,
                    delta * (abs_residuals - 0.5 * delta))
    loss = np.sum(loss) / n
    
    return loss

def huber_loss_grad(params, X, y, delta=1.0):
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    grad = -np.dot(X.T, np.where(abs_residuals <= delta,
                                 residuals,
                                 delta * np.sign(residuals))) / n

    return grad

# Initialize the loop variables
warnings.filterwarnings('ignore')
condition_met = False
start_time = time.time()
time_limit = 30
iteration = 0

# Define constraints
constraints = {
    'Tempo abaixo de 10 mbar(min)': (9.650573007040464, 23),
    'Vazão média de argônio durante o sopro de O2(Nm3/h)': (3.5215818863059383, 6.8251891508760565),
    'Pressão média de vácuo durante o sopro de O2(mbar)': (400, 693.3164964745301),
    'Volume de O2 soprado(m3)': (115.39118253104017, 248.45226169596572),
    'Vazão média de argônio durante a descarburação(Nm3/h)': (4.55, 7.87)
}

while not condition_met:
    try:
        # Track elapsed time
        elapsed_time = time.time() - start_time
        if elapsed_time > time_limit:
            print("Time limit exceeded. Stopping the loop.")
            break

        iteration += 1
        print(f"Iteration: {iteration}, Time elapsed: {elapsed_time:.2f} seconds")

        # Reset index for contr DataFrame
        contr.reset_index(drop=True, inplace=True)

        # Concatenate and process DataFrames
        tt2 = pd.concat([ent_, contr], axis=1)
        tt2.fillna(0, inplace=True)
        tt2 = pd.DataFrame(tt2.sum()).T
        
        # Apply constraints efficiently
        for col, (a_min, a_max) in constraints.items():
            if col in tt2.columns:
                tt2[col] = np.clip(tt2[col], a_min=a_min, a_max=a_max)

        required_columns = [
            'Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
            'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
            'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
            'Temperatura do aço antes do VOD(°C)', 'Peso de aço antes do VOD(kg) ',
            'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela',
            'Vazão média de argônio durante o sopro de O2(Nm3/h)',
            'Pressão média de vácuo durante o sopro de O2(mbar)',
            'Tempo abaixo de 10 mbar(min)',
            'Vazão média de argônio durante a descarburação(Nm3/h)',
            'Volume de O2 soprado(m3)'
        ]

        tt2_test = tt2[required_columns]

        x_ = dt[required_columns]
        y_ = dt['opt']

        x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_, y_, test_size=0.9)

        # Optimize using Huber loss without L2 regularization
        initial_params = np.zeros(x_train_3.shape[1])
        result = minimize(fun=huber_loss, x0=initial_params, args=(x_train_3.values, y_train_3.values),
                          jac=huber_loss_grad, method='BFGS')

        optimal_params = result.x

        # Predicting on tt2
        tt2_features = tt2_test[required_columns].values
        y_pred_tt2_prob = 1 / (1 + np.exp(-np.dot(tt2_features, optimal_params)))
        y_pred_tt2_binary = (y_pred_tt2_prob >= 0.5).astype(int)

        # Display predictions for tt2
        tt2['Predicted_Probability'] = y_pred_tt2_prob
        tt2['Predicted_Class'] = y_pred_tt2_binary

        print("Predictions on tt2:")
        print(tt2[['Predicted_Probability', 'Predicted_Class']])

        # Train Naive Bayes model on the predicted probabilities
        x_train_nb = np.column_stack((x_train_3.values, 1 / (1 + np.exp(-np.dot(x_train_3.values, optimal_params)))))
        x_test_nb = np.column_stack((x_test_3.values, 1 / (1 + np.exp(-np.dot(x_test_3.values, optimal_params)))))

        nb_model = GaussianNB()
        nb_model.fit(x_train_nb, y_train_3)  # Fit on the combined data

        # Make predictions using the Naive Bayes model
        y_pred_nb_prob = nb_model.predict_proba(np.column_stack((tt2_test[required_columns].values, y_pred_tt2_prob)))[:, 1]
        y_pred_nb_binary = (y_pred_nb_prob >= 0.5).astype(int)

        # Display Naive Bayes predictions for tt2
        tt2['Naive_Bayes_Probability'] = y_pred_nb_prob
        tt2['Naive_Bayes_Class'] = y_pred_nb_binary

        print("Naive Bayes Predictions on tt2:")
        print(tt2[['Naive_Bayes_Probability', 'Naive_Bayes_Class']])

        # Fit the Logit model from Statsmodels
        logit_model = sm.Logit(y_train_3, x_train_3)
        results = logit_model.fit(method='bfgs', cov_type='HC3')

        # Extract coefficients and compute exponentiated coefficients
        co = results.params
        exp_coeff = np.exp(co) / (np.exp(co) + 1)
        exp_coeff_df = pd.DataFrame({'exp_coeff': exp_coeff})

        # Extract results summary
        results_summary = results.summary()
        results_as_html = results_summary.tables[1].as_html()
        df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
        df_results['exp_coeff'] = exp_coeff_df

        # Filter significant variables
        impact_var2 = df_results[df_results['P>|z|'] < 0.05]

        # Calculate results
        results_list = []
        for i in impact_var2.index:
            value = impact_var2.loc[i, 'exp_coeff']
            if i in constraints:
                correction = tt2_test[i].median()
                result = (value - 0.5) * correction if value != 0.5 else 0
                results_list.append({'Column': i, 'Alteração': result})

        results_df_ = pd.DataFrame(results_list).T
        results_df_.columns = results_df_.iloc[0]
        results_df_ = results_df_[1:]

        # Common columns
        common_columns = list(set(constraints.keys()) & set(results_df_.columns))
        contr_common = contr[common_columns]
        results_common = results_df_[common_columns]

        # Combine and update
        combined_df = pd.concat([contr_common, results_common], ignore_index=True)
        final = combined_df.sum().reset_index()
        final_df = final.T
        final_df.columns = final_df.iloc[0]
        final_df = final_df[1:]

        contr.update(final_df[common_columns])
        contr.reset_index(drop=True, inplace=True)

        # Garbage collection
        gc.collect()

        # Check condition with debugging prints
        print(f"Predicted probabilities: {y_pred_nb_prob}")
        print(f"Predicted classes: {y_pred_nb_binary}")
        print("Checking stopping condition...")

        # Adjusted stopping condition
        if np.mean(y_pred_nb_prob) >= 0.99 and np.all(y_pred_nb_binary == 1):
            condition_met = True

    except Exception as e:
        print(f"An error occurred: {e}")
        continue


Iteration: 1, Time elapsed: 0.00 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0               0.999998                1
Naive Bayes Predictions on tt2:
   Naive_Bayes_Probability  Naive_Bayes_Class
0                 0.839989                  1
         Current function value: 0.369817
         Iterations: 35
         Function evaluations: 47
         Gradient evaluations: 39
Predicted probabilities: [0.83998885]
Predicted classes: [1]
Checking stopping condition...
Iteration: 2, Time elapsed: 0.29 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0          1.076566e-222                0
Naive Bayes Predictions on tt2:
   Naive_Bayes_Probability  Naive_Bayes_Class
0                 0.936191                  1
         Current function value: 0.345300
         Iterations: 35
         Function evaluations: 50
         Gradient evaluations: 41
An error occurred: single positional indexer is out-of-bounds
Iteration: 3, Time elapsed: 0.33 seconds

In [586]:
#NAIVE_BAYES:
#Primeira volta:
# Initialize the Gaussian Naive Bayes classifier
# Required columns for the model
warnings.filterwarnings('ignore')

required_columns = [
    'Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
    'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
    'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
    'Temperatura do aço antes do VOD(°C)','Peso de aço antes do VOD(kg) ',
    'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela',
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
    'Pressão média de vácuo durante o sopro de O2(mbar)',
    'Tempo abaixo de 10 mbar(min)',
    'Vazão média de argônio durante a descarburação(Nm3/h)',
    'Volume de O2 soprado(m3)']

# Define control DataFrame
columns_control = [
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
    'Pressão média de vácuo durante o sopro de O2(mbar)',
    'Tempo abaixo de 10 mbar(min)',
    'Vazão média de argônio durante a descarburação(Nm3/h)',
    'Volume de O2 soprado(m3)'
]

# Example control values
stp = [[ 5.  , 494.7 ,  18.7 ,   6.54, 194.  ]]
contr = pd.DataFrame(stp, columns=columns_control)

# Prepare input DataFrame
ent = teste[['Teor de C antes do VOD(%)',
       'Teor de Cr antes do VOD(%)', 'Teor de Si antes do VOD(%)',
       'Teor de Mn antes do VOD(%)', 'Teor de Ni antes do VOD(%)','Teor de Mo antes do VOD(%)',
          'Temperatura do aço antes do VOD(°C)',
       'Peso de aço antes do VOD(kg) ', 'Borda livre da panela(mm)',
       'Vida de plug da panela', 'Vida da panela']]

#ent_ = ent.sample(n=1)

# Combine control and input data
contr.reset_index(drop=True, inplace=True)
tt2 = pd.concat([ent_, contr], axis=1)
tt2.fillna(0, inplace=True)
tt2 = pd.DataFrame(tt2.sum()).T
# Predicting on tt2

tt2_features = tt2.values
# Prepare test data for predictions
tt2_test = tt2[columns_control]

# Prepare Data for training
x_ = dt[required_columns]
y_ = dt['opt']
scaler = RobustScaler()
x_scaled = scaler.fit_transform(x_)

# Split the data

gnb = GaussianNB()
#x_tr, x_tes, y_tr, y_tes = train_test_split(x_, y_, test_size=0.2, random_state=42)
# Fit the model on the training data
gnb.fit(x_ , y_)

# Make predictions on the test data
y_pred = gnb.predict(tt2_features)
y_prob = gnb.predict_proba(tt2_features)
print(y_pred, ent_.index, y_prob)

[0] Index([39], dtype='int64') [[0.61593032 0.38406968]]


In [596]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from sklearn.naive_bayes import GaussianNB
import time
import warnings
import gc

# Custom Huber loss function for logistic regression without L2 regularization
def huber_loss(params, X, y, delta=1.0):
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    loss = np.where(abs_residuals <= delta,
                    0.5 * residuals**2,
                    delta * (abs_residuals - 0.5 * delta))
    loss = np.sum(loss) / n
    
    return loss

def huber_loss_grad(params, X, y, delta=1.0):
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    grad = -np.dot(X.T, np.where(abs_residuals <= delta,
                                 residuals,
                                 delta * np.sign(residuals))) / n

    return grad

# Initialize the loop variables
warnings.filterwarnings('ignore')
condition_met = False
start_time = time.time()
time_limit = 30
iteration = 0

# Define constraints
constraints = {
    'Tempo abaixo de 10 mbar(min)': (9.650573007040464, 23),
    'Vazão média de argônio durante o sopro de O2(Nm3/h)': (3.5215818863059383, 6.8251891508760565),
    'Pressão média de vácuo durante o sopro de O2(mbar)': (400, 693.3164964745301),
    'Volume de O2 soprado(m3)': (115.39118253104017, 248.45226169596572),
    'Vazão média de argônio durante a descarburação(Nm3/h)': (4.55, 7.87)
}

while not condition_met:
    try:
        # Track elapsed time
        elapsed_time = time.time() - start_time
        if elapsed_time > time_limit:
            print("Time limit exceeded. Stopping the loop.")
            break

        iteration += 1
        print(f"Iteration: {iteration}, Time elapsed: {elapsed_time:.2f} seconds")

        # Reset index for contr DataFrame
        contr.reset_index(drop=True, inplace=True)

        # Concatenate and process DataFrames
        tt2 = pd.concat([ent_, contr], axis=1)
        tt2.fillna(0, inplace=True)
        tt2 = pd.DataFrame(tt2.sum()).T
        
        # Apply constraints efficiently
        for col, (a_min, a_max) in constraints.items():
            if col in tt2.columns:
                tt2[col] = np.clip(tt2[col], a_min=a_min, a_max=a_max)

        required_columns = [
            'Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
            'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
            'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
            'Temperatura do aço antes do VOD(°C)', 'Peso de aço antes do VOD(kg) ',
            'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela',
            'Vazão média de argônio durante o sopro de O2(Nm3/h)',
            'Pressão média de vácuo durante o sopro de O2(mbar)',
            'Tempo abaixo de 10 mbar(min)',
            'Vazão média de argônio durante a descarburação(Nm3/h)',
            'Volume de O2 soprado(m3)'
        ]

        tt2_test = tt2[required_columns]

        x_ = dt[required_columns]
        y_ = dt['opt']

        x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_, y_, test_size=0.9)

        # Optimize using Huber loss without L2 regularization
        initial_params = np.zeros(x_train_3.shape[1])
        result = minimize(fun=huber_loss, x0=initial_params, args=(x_train_3.values, y_train_3.values),
                          jac=huber_loss_grad, method='BFGS')

        optimal_params = result.x

        # Predicting on tt2
        tt2_features = tt2_test[required_columns].values
        y_pred_tt2_prob = 1 / (1 + np.exp(-np.dot(tt2_features, optimal_params)))
        y_pred_tt2_binary = (y_pred_tt2_prob >= 0.5).astype(int)

        # Display predictions for tt2
        tt2['Predicted_Probability'] = y_pred_tt2_prob
        tt2['Predicted_Class'] = y_pred_tt2_binary

        print("Predictions on tt2:")
        print(tt2[['Predicted_Probability', 'Predicted_Class']])

        # Train Naive Bayes model on the predicted probabilities
        x_train_nb = np.column_stack((x_train_3.values, 1 / (1 + np.exp(-np.dot(x_test_3.values, optimal_params)))))
        x_test_nb = np.column_stack((x_test_3.values, 1 / (1 + np.exp(-np.dot(x_test_3.values, optimal_params)))))

        nb_model = GaussianNB()
        scaler = RobustScaler()
        x_s = scaler.fit(x_)
        nb_model.fit(x_, y_)  # Fit on the combined data

        # Make predictions using the Naive Bayes model
        y_pred_nb_prob = nb_model.predict_proba(tt2_features)[:, 1]
        y_pred_nb_binary = (y_pred_nb_prob >= 0.5).astype(int)

        # Display Naive Bayes predictions for tt2
        tt2['Naive_Bayes_Probability'] = y_pred_nb_prob
        tt2['Naive_Bayes_Class'] = y_pred_nb_binary

        print("Naive Bayes Predictions on tt2:")
        print(tt2[['Naive_Bayes_Probability', 'Naive_Bayes_Class']])

        # Fit the Logit model from Statsmodels
        logit_model = sm.Logit(y_train_3, x_train_3)
        results = logit_model.fit(method='bfgs', cov_type='HC2')

        # Extract coefficients and compute exponentiated coefficients
        co = results.params
        exp_coeff = np.exp(co) / (np.exp(co) + 1)
        exp_coeff_df = pd.DataFrame({'exp_coeff': exp_coeff})

        # Extract results summary
        results_summary = results.summary()
        results_as_html = results_summary.tables[1].as_html()
        df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
        df_results['exp_coeff'] = exp_coeff_df

        # Filter significant variables
        impact_var2 = df_results[df_results['P>|z|'] < 0.05]

        # Calculate results
        results_list = []
        for i in impact_var2.index:
            value = impact_var2.loc[i, 'exp_coeff']
            if i in constraints:
                correction = tt2_test[i].median()
                result = (value - 0.5) * correction if value != 0.5 else 0
                results_list.append({'Column': i, 'Alteração': result})

        results_df_ = pd.DataFrame(results_list).T
        results_df_.columns = results_df_.iloc[0]
        results_df_ = results_df_[1:]

        # Common columns
        common_columns = list(set(constraints.keys()) & set(results_df_.columns))
        contr_common = contr[common_columns]
        results_common = results_df_[common_columns]

        # Combine and update
        combined_df = pd.concat([contr_common, results_common], ignore_index=True)
        final = combined_df.sum().reset_index()
        final_df = final.T
        final_df.columns = final_df.iloc[0]
        final_df = final_df[1:]

        contr.update(final_df[common_columns])
        contr.reset_index(drop=True, inplace=True)

        # Garbage collection
        gc.collect()

        # Check condition with debugging prints
        print(f"Predicted probabilities: {y_pred_nb_prob}")
        print(f"Predicted classes: {y_pred_nb_binary}")
        print("Checking stopping condition...")

        # Adjusted stopping condition
        if np.mean(y_pred_nb_prob) >= 0.99 and np.all(y_pred_nb_binary == 1):
            condition_met = True

    except Exception as e:
        print(f"An error occurred: {e}")
        continue


Iteration: 1, Time elapsed: 0.00 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0                    0.0                0
An error occurred: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 51 and the array at index 1 has size 460
Iteration: 2, Time elapsed: 0.03 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0          6.325301e-226                0
An error occurred: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 51 and the array at index 1 has size 460
Iteration: 3, Time elapsed: 0.06 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0           3.667066e-78                0
An error occurred: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 51 and the a

In [608]:
print(y_pred_nb_prob, ent_.index)

[0.99797497] Index([39], dtype='int64')


In [609]:
tt2[columns_control]

,Vazão média de argônio durante o sopro de O2(Nm3/h),Pressão média de vácuo durante o sopro de O2(mbar),Tempo abaixo de 10 mbar(min),Vazão média de argônio durante a descarburação(Nm3/h),Volume de O2 soprado(m3)
0,5.0,494.7,20.849486,7.510126,191.453191


In [610]:
stp

[[5.0, 494.7, 18.7, 6.54, 194.0]]